In [80]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import re
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
from nltk.tokenize import word_tokenize

from function import *

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nicolaibernsen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nicolaibernsen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Scraping KU, AU and AAU

In [81]:
ku_list = []

search_list = ['psykologi', 'sociologi', 'statskundskab', 'antropologi', 'oekonomi']


for k in search_list:
    soup = extract_ku(k)
    ku_list.append(transform_ku(soup))

ku_df = pd.DataFrame(data=ku_list)

#Merging the two colums 
ku_df=ku_df[0]+ku_df[1]

#Making the object a dataframe
ku_df=pd.DataFrame(ku_df)

ku_df=pd.DataFrame.transpose(ku_df)

ku_df.columns=['cand.psych_ku', 'cand.scient.soc_ku', 'cand.scient.pol_ku', 'cand.scient.anth_ku', 'cand.oecon_ku']

In [82]:
soup = extract_au('statskundskab')

divs = soup.find_all("div", class_="large-8 medium-8 medium-only-portrait-12 small-12 columns")

text = soup.find_all('p')

text_stats = text[75:77]

stats_df = pd.DataFrame(data=text_stats, columns=['cand.scient.pol_au'])
stats = pd.DataFrame([', '.join(stats_df['cand.scient.pol_au'].to_list())], columns=['cand.scient.pol_au'])

In [83]:
soup = extract_au('oekonomi')

divs = soup.find_all("div", class_="large-8 medium-8 medium-only-portrait-12 small-12 columns")

text = soup.find_all('p')

text_oek = text[60:62]

oek_df = pd.DataFrame(data=text_oek, columns=['cand.oecon_au'])
oek = pd.DataFrame([', '.join(oek_df['cand.oecon_au'].to_list())], columns=['cand.oecon_au'])

In [84]:
soup = extract_au('antropologi')

divs = soup.find_all("div", class_="large-8 medium-8 medium-only-portrait-12 small-12 columns")

text = soup.find_all('p')

text_ant = text[50:55]

ant_df = pd.DataFrame(data=text_ant, columns=['cand.scient.anth_au'])
ant = pd.DataFrame([', '.join(ant_df['cand.scient.anth_au'].to_list())], columns=['cand.scient.anth_au'])

In [85]:
soup = extract_au('psykologi')

divs = soup.find_all("div", class_="large-8 medium-8 medium-only-portrait-12 small-12 columns")

text = soup.find_all('p')

text_psyk = text[74:78]

psyk_df = pd.DataFrame(data=text_psyk, columns=['cand.psych_au'])
psyk = pd.DataFrame([', '.join(psyk_df['cand.psych_au'].to_list())], columns=['cand.psych_au'])

In [86]:
frames = [ant, stats, psyk, oek]

au_df = pd.concat(frames, axis=1)

au_df_list = au_df['cand.scient.anth_au'].to_list()
au_list = []

for i in au_df_list:
    au_list.append(clean_text(i))

au_list

['dine jobmuligheder som antropolog afhænger meget af dine valg og interesser under studiet samt hvilket tilvalg og eventuel kandidatuddannelse du vælger på bacheloruddannelsen i antropologi får du indblik i kulturforskelle og ligheder og lærer at analysere vanskelige kulturspørgsmål du bliver trænet i at udvælge og anvende antropologiske metoder og lærer at formidle dine resultater skriftligt og mundtligt undervejs har du mulighed for at specialisere dig i bestemte emner og regioner såsom sundhed miljø integration eller i særlige kulturelle problemstillinger uddannede antropologer arbejder typisk som projektledere konsulenter undervisere eller som analyse og evalueringsmedarbejdere det kunne eksempelvis være indenfor innovation det sociale område eller udvikling og ulandsbistand ']

In [87]:
def transform_aau(soup):

    divs = soup.find_all("main", class_="Main_Main__2KIvG")

    for item in divs:
        text_aau = item.find_all()[0].text.strip()

        aau_text = {
            "text_aau" : text_aau, 
        }
        aau_list.append(aau_text)

        text_aau = clean_text(text_aau)
        
    return text_aau

aau_list = []

search_list = ['psykologi', 'sociologi', 'oekonomi']

for k in search_list:
    try: 
        soup = extract_aau(k)
        transform_aau(soup)
    except:
        break

aau_df = pd.DataFrame(data=aau_list)

aau_df=pd.DataFrame.transpose(aau_df)

aau_df.columns=['cand.psych_aau', 'cand.scient.soc_aau', 'cand.oecon_aau']

aau_df = aau_df.reset_index(drop=True)

aau_df

,cand.psych_aau,cand.scient.soc_aau,cand.oecon_aau
0,KandidatPsykologiUddannelsen uddanner dig til ...,KandidatSociologiUddannelsen bygger videre på ...,KandidatØkonomiDu lærer at analysere samfundsø...


## Combining Dataframes

In [88]:
merge_frames = [ku_df, au_df, aau_df]

combined_df = pd.concat(merge_frames, axis=1)

combined_df['cand.psych_ku_string'] = [','.join(map(str, l)) for l in combined_df['cand.psych_ku']]
combined_df['cand.scient.pol_ku_string'] = [','.join(map(str, l)) for l in combined_df['cand.scient.pol_ku']]
combined_df['cand.oecon_ku_string'] = [','.join(map(str, l)) for l in combined_df['cand.oecon_ku']]
combined_df['cand.scient.anth_ku_string'] = [','.join(map(str, l)) for l in combined_df['cand.scient.anth_ku']]
combined_df['cand.scient.soc_ku_string'] = [','.join(map(str, l)) for l in combined_df['cand.scient.soc_ku']]

combined_df['cand.psych'] = combined_df['cand.psych_aau'] + combined_df['cand.psych_au'] + combined_df['cand.psych_ku_string']
combined_df['cand.scient.anth'] = combined_df['cand.scient.anth_au'] + combined_df['cand.scient.anth_ku_string']
combined_df['cand.scient.pol'] = combined_df['cand.scient.pol_au'] + combined_df['cand.scient.pol_ku_string']
combined_df['cand.scient.soc'] = combined_df['cand.scient.soc_aau'] + combined_df['cand.scient.soc_ku_string']
combined_df['cand.oecon'] = combined_df['cand.oecon_aau'] + combined_df['cand.oecon_au'] + combined_df['cand.oecon_ku_string']

In [89]:
combined_df.drop(['cand.psych_aau', 'cand.scient.soc_aau', 'cand.oecon_aau', 'cand.psych_au', \
                  'cand.scient.anth_au', 'cand.scient.pol_au','cand.oecon_au', \
                  'cand.psych_ku', 'cand.scient.soc_ku', 'cand.scient.pol_ku', 'cand.scient.anth_ku', \
                  'cand.oecon_ku', 'cand.psych_ku_string', 'cand.scient.pol_ku_string', 'cand.oecon_ku_string', \
                  'cand.scient.anth_ku_string', 'cand.scient.soc_ku_string'], axis=1, inplace=True)

In [90]:
university_df = combined_df
university_df

,cand.psych,cand.scient.anth,cand.scient.pol,cand.scient.soc,cand.oecon
0,KandidatPsykologiUddannelsen uddanner dig til ...,Dine jobmuligheder som antropolog afhænger meg...,Som bachelor i Statskundskab bliver du først o...,KandidatSociologiUddannelsen bygger videre på ...,KandidatØkonomiDu lærer at analysere samfundsø...


## Scraping UG

In [91]:
search_list = ['cand.psych', 'cand.oecon', 'cand.scient.pol', 'cand.scient.anth', 'cand.scient.soc']

education_url = []

for k in search_list:
    for i in range(1):
        try: 
            education_url.append(UG(i,k))
        except:
            break
        
education_url[1].pop(0)
del education_url[4][0:3]

In [92]:
psych = clean_text(extract_UG(education_url[0][0]))
oecon = extract_UG(education_url[1][0])
pol = extract_UG(education_url[2][0])
anth = extract_UG(education_url[3][0])
soc = extract_UG(education_url[4][0])

## Combining skills from UG and Universities

In [93]:
strings_psych = psych + university_df['cand.psych']
strings_oecon = oecon + university_df['cand.oecon']
strings_pol = pol + university_df['cand.scient.pol']
strings_anth = anth + university_df['cand.scient.anth']
strings_soc = soc + university_df['cand.scient.soc']


psych_comb = " ".join(strings_psych)
oecon_comb = " ".join(strings_oecon)
pol_comb = " ".join(strings_pol)
anth_comb = " ".join(strings_anth)
soc_comb = " ".join(strings_soc)

In [94]:
psych_series = pd.Series(psych_comb)
oecon_series = pd.Series(oecon_comb)
pol_series = pd.Series(pol_comb)
anth_series = pd.Series(anth_comb)
soc_series = pd.Series(soc_comb)

psych_df = pd.DataFrame(psych_series)
oecon_df = pd.DataFrame(oecon_series)
pol_df = pd.DataFrame(pol_series)
anth_df = pd.DataFrame(anth_series)
soc_df = pd.DataFrame(soc_series)

psych_df['cand.psych'] = psych_df[0]
oecon_df['cand.oecon'] = oecon_df[0]
pol_df['cand.scient.pol'] = pol_df[0]
anth_df['cand.scient.anth'] = anth_df[0]
soc_df['cand.scient.soc'] = soc_df[0]

In [104]:
final_df = pd.concat([psych_df['cand.psych'], oecon_df['cand.oecon'], pol_df['cand.scient.pol'], \
                      anth_df['cand.scient.anth'], soc_df['cand.scient.soc']], axis=1)
final_df


# for column in final_df[column]:
    # for text in column:
        # clean_text(text)

,cand.psych,cand.oecon,cand.scient.pol,cand.scient.anth,cand.scient.soc
0,på kandidatuddannelsen arbejder du videnskabe...,på kandidatuddannelsen får du en udvidet fors...,du får analytiske færdigheder og teoretisk in...,du fortsætter studiet af menneskers sociale o...,du vil herunder få specialiseret viden om de ...


In [105]:
final_df

,cand.psych,cand.oecon,cand.scient.pol,cand.scient.anth,cand.scient.soc
0,på kandidatuddannelsen arbejder du videnskabe...,på kandidatuddannelsen får du en udvidet fors...,du får analytiske færdigheder og teoretisk in...,du fortsætter studiet af menneskers sociale o...,du vil herunder få specialiseret viden om de ...


In [97]:
for column in final_df.columns:
    for text in final_df[column]:
        print(text)

 på kandidatuddannelsen arbejder du videnskabeligt med faget og øver dig i at bruge det du lærer i praksis uddannelsen belyser forskellige psykologiske problemstillinger som du lærer at tage stilling til gennem teoretisk og metodisk orienteret undervisning som færdiguddannet cand psych kan du bl a arbejde inden for social og sundhedsområdet du kan også arbejde som praktiserende psykolog eller blive ansat i private virksomheder hvor du kan arbejde med personaleudvikling evaluering og samarbejde der er endvidere mulighed for at undervise og forske på uddannelsen lærer du om teorier der beskæftiger sig med menneskets bevidsthedsliv og erkendelse den menneskelige personligheds struktur og udvikling psykiske vanskeligheder lidelser og livskriser du lærer også om interaktionen mellem det enkelte menneske og andre mennesker og samfundet kandidatuddannelsen indeholder undervisning i fag som fx arbejds og organisationspsykologi klinisk psykologi pædagogisk psykologi kognitions og udviklingspsyk

In [ ]:
final_df['cand.psych'] = final_df['cand.psych'].apply(word_tokenize)
final_df['cand.oecon'] = final_df['cand.oecon'].apply(word_tokenize)
final_df['cand.scient.pol'] = final_df['cand.scient.pol'].apply(word_tokenize)
final_df['cand.scient.anth'] = final_df['cand.scient.anth'].apply(word_tokenize)
final_df['cand.scient.soc'] = final_df['cand.scient.soc'].apply(word_tokenize)

In [ ]:
stop = stopwords.words('danish')
final_df['cand.psych'] = final_df['cand.psych'].apply(lambda words: [word for word in words if word not in stop])
final_df['cand.oecon'] = final_df['cand.oecon'].apply(lambda words: [word for word in words if word not in stop])
final_df['cand.scient.pol'] = final_df['cand.scient.pol'].apply(lambda words: [word for word in words if word not in stop])
final_df['cand.scient.anth'] = final_df['cand.scient.anth'].apply(lambda words: [word for word in words if word not in stop])
final_df['cand.scient.soc'] = final_df['cand.scient.soc'].apply(lambda words: [word for word in words if word not in stop])

In [ ]:
final_df.to_csv('out.csv')
